In [8]:
import pandas as pd
import numpy as np
import os

In [9]:
# load files
files = os.listdir("../Datasets/")

In [242]:
class Processing:

    ARCHIVOSORIGINALES = ['BD tesis DEPURADA ESP 2013-2018.xlsx',
    'Resultados descentralizadas pequeñas_2018.xlsx',
    'ResultadosFinalesFURAGV2019.xlsx',
    'ResultadosFURAG_2018.xlsx',
    'ResultadosMDI_vig2020.xlsx',
    'Resultados_Desempeno_v2021.xlsx']

    def __init__(self,file_list):
        self.file_list = file_list
    
    def read_files(self):
        df_list = list()

        for file in self.file_list:
            df = pd.ExcelFile("../Datasets/"+file)
            if (len(df.sheet_names) > 1) & (file==Processing.ARCHIVOSORIGINALES[0]):
                # final_df = pd.DataFrame()
                final_df = list()
                for sheets in df.sheet_names:

                    sheet_df = df.parse(sheets)
                    final_colnames = sheet_df.iloc[0]
                    final_colnames.replace(np.nan,"Empresa",inplace=True)                     
                    sheet_df.columns = final_colnames
                    sheet_df.drop(sheet_df.index[0],inplace=True)
                    sheet_df["year"] = sheets
                    sheet_df.set_index(sheet_df["year"]+sheet_df.index.astype(str),inplace=True)

                    if sheets != "2013":
                        # add a new column
                        sheet_df["Otros Activos Pesos"] = 0
                    
                    elif sheets =="2013":
                        sheet_df["Otros Pasivos Pesos"] = 0

                    final_df.append(sheet_df)
                    # final_df = pd.concat([final_df,sheet_df],ignore_index=True)
                
            else:
                final_df = df.parse(0)      
                              
            df_list.append(final_df)

        self.df_list = df_list

In [243]:
datasets_processing = Processing(files)
datasets_processing.read_files()

In [264]:
datasets_processing.df_list[0][1].sort_index(axis=1,inplace=True)
datasets_processing.df_list[0][0].sort_index(axis=1,inplace=True)

In [270]:
datasets_processing.df_list[0][2].drop_duplicates()

,Empresa,Activo Corriente,Activo No Corriente,Total Activo,Pasivo Corriente,Pasivo No Corriente,Total Pasivo,Patrimonio,Efectivo,Efectivo Porcentaje,...,Gastos de Administración,Provisiones Agotamientos Depreciaciones y Amortizaciones,Utilidad Operacional,Otros Ingresos,Otros Gastos,Utilidad Antes de Ajuste,Corrección Monetaria,Utilidad Neta,year,Otros Activos Pesos
20151,ADMINISTRACION PUBLICA COOPERATIVA DE SERVICIO...,6297610,2954230,9251840,4137612,0,4137612,5114228,0,0,...,19874158,1338596,-24000574,126,779132,-24779580,0,-24779580,2015,0
20152,ALCALDIA MUNICIPAL DEL TAMBO NARIÑO,7046788880,0,7046788880,51857340,0,51857340,6994931540,26062554,59.517312,...,75287312,901000,-63723282,335193460,1900000,269570178,0,269570178,2015,0
20153,ASOCIACION DE USUARIOS DEL ACUEDUCTO Y ALCANT...,48666737,1239336348,1288003085,15832211,0,15832211,1272170874,37875748,77.826767,...,78061829,0,2532149,113656,2581675,64130,0,64130,2015,0
20154,ASOCIACION DE SOCIOS DEL ACUEDUCTO CAMARGO,71139307,82272654,153411961,10978856,0,10978856,142433105,71139133,99.999755,...,34482980,6468857,589566,3134509,2941514,782561,0,782561,2015,0
20155,ASOCIACION DE SOCIOS DEL ACUEDUCTO EL CERRO SA...,220774963,2296756722,2517531685,24443586,290571633,315015219,2202516466,29875266,13.531999,...,120556441,50632325,211476299,27297171,47932087,190841383,0,190841383,2015,0
20156,EMPRESA DE ACUEDUCTO ALCANTARILLADO ASEO Y SER...,23734425,11895413,35629838,10379052,0,10379052,25250786,10146209,42.748914,...,25825525,0,8921447,0,0,8921447,0,8921447,2015,0
20157,"EMPRESA DE ACUEDUCTO, ALCANTARILLADO Y ASEO DE...",3262430696,16733833,3279164529,3883555803,0,3883555803,-604391274,2237419550,68.581366,...,303082485,1887044,-377541860,35202,4264458,-381771116,0,-381771116,2015,0
20158,EMPRESA DE AGUA POTABLE Y SANEAMIENTO BASICO D...,1461966667,15963156294,17425122961,4679073596,0,4679073596,12746049365,59478003,4.068356,...,205858525,1091251877,2096129,775733484,4859601,772970012,0,772970012,2015,0
20159,EMPRESA DE SERVICIOS PUBLICOS DE FALAN S.A.S E...,34384053,675172,35059225,7751413,0,7751413,27307812,7302993,21.239477,...,0,652808,18856923,0,0,18856923,0,18856923,2015,0
201510,EMPRESA DE SERVICIOS PUBLICOS DEL VALLE DE SAN...,311619523,139279134,450898657,359758083,0,359758083,91140574,99863514,32.046617,...,69611800,9667431,-52779897,32930561,116600,-19965936,0,-19965936,2015,0


In [233]:
list(set(datasets_processing.df_list[0][1].columns.tolist())-set(datasets_processing.df_list[0][0].columns.tolist()))

['Otros Pasivos Pesos']